### Procesamiento **ETL** al archivo **`user_reviews.json`**

In [1]:
# Importamos las librerias necesarias para el procesamiento de los datos
import pandas as pd
import numpy as np
import json
import ast
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [8]:
filas = list()
with open(r"data\australian_user_reviews.json", "r",encoding="Latin-1") as archivo:
    for linea in archivo.readlines():
        filas.append(ast.literal_eval(linea))

df_user_reviews = pd.DataFrame(filas)

In [9]:
df_user_reviews.info()
df_user_reviews.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


(25799, 3)

In [10]:
df_user_reviews.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

### Limpieza de la columna **``Reviews``**

In [11]:
def desanidar_reviews_y_limpiar(df):
    # Desanidar el diccionario de la columna 'reviews'
    df_expanded_reviews = df['reviews'].explode().apply(pd.Series)

    # Crear un nuevo DataFrame combinando columnas originales y desanidadas
    df_combined = pd.concat([df.drop(['reviews'], axis=1), df_expanded_reviews], axis=1)

    # Eliminar filas duplicadas basadas en la columna 'review'
    df_result = df_combined.drop_duplicates(subset='review', keep='first')

    # Eliminar la última columna redundante
    df_result = df_result.iloc[:, :-1]

    return df_result

# Llamada a la función con tu DataFrame específico
df_user_reviews = desanidar_reviews_y_limpiar(df_user_reviews)

In [12]:
df_user_reviews

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,130,No ratings yet,True,if you liked Half life i would really recommen...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...


In [13]:
df_user_reviews.info()
df_user_reviews.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 55314 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      55314 non-null  object
 1   user_url     55314 non-null  object
 2   funny        55313 non-null  object
 3   posted       55313 non-null  object
 4   last_edited  55313 non-null  object
 5   item_id      55313 non-null  object
 6   helpful      55313 non-null  object
 7   recommend    55313 non-null  object
 8   review       55313 non-null  object
dtypes: object(9)
memory usage: 4.2+ MB


user_id        0
user_url       0
funny          1
posted         1
last_edited    1
item_id        1
helpful        1
recommend      1
review         1
dtype: int64

In [14]:
# Filtramos las filas donde 'item_id', 'recommend' y 'review' no son nulos
df_user_reviews = df_user_reviews[df_user_reviews['item_id'].notnull() & 
                                  df_user_reviews['recommend'].notnull() & 
                                  df_user_reviews['review'].notnull()]

In [15]:
#eliminamos "funny", "last edited y "helpful"
df_user_reviews.drop(['funny', 'last_edited', 'helpful'], axis=1, inplace=True)
df_user_reviews.isna().sum()

C:\Users\57301\AppData\Local\Temp\ipykernel_14104\2199371940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_reviews.drop(['funny', 'last_edited', 'helpful'], axis=1, inplace=True)


user_id      0
user_url     0
posted       0
item_id      0
recommend    0
review       0
dtype: int64

### Limpieza de la columna **``posted``**

In [16]:
# Extraer el año de la columna 'posted' y almacenarlo en una nueva columna 'year'
df_user_reviews.loc[:, 'year'] = df_user_reviews['posted'].str.extract(r'(\d{4})')

# Eliminar la columna 'posted' del DataFrame df_reviews_user
df_user_reviews = df_user_reviews.drop('posted', axis=1)

C:\Users\57301\AppData\Local\Temp\ipykernel_14104\308294874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_reviews.loc[:, 'year'] = df_user_reviews['posted'].str.extract(r'(\d{4})')


In [17]:
unique_years = df_user_reviews['year'].unique()
unique_years

array(['2011', '2014', '2013', nan, '2015', '2012', '2010'], dtype=object)

Estos porcentajes indican la distribución de los años en la columna **`year`**. se puedes observar que la mayoría de los registros están concentrados en los años **`2014`** y **`2015`**, mientras que los años anteriores tienen una presencia más baja. Esta información es útil para entender la composición temporal de los datos y es importante para el posterior análisis exploratorio que se realizara. 



Se puede interpretar que:



* Para el año 2014, el 44.91% de los registros tienen ese año.
* Para el año 2015, el 37.70% de los registros tienen ese año.
* Para el año 2013, el 13.69% de los registros tienen ese año.
* Para el año 2012, el 2.45% de los registros tienen ese año.
* Para el año 2011, el 1.11% de los registros tienen ese año.
* Para el año 2010, el 0.14% de los registros tienen ese año.

In [18]:
# Calcular los value counts de la columna 'year'
year_counts = df_user_reviews['year'].value_counts(normalize=True)
year_counts

year
2014    0.449068
2015    0.376969
2013    0.136931
2012    0.024506
2011    0.011085
2010    0.001440
Name: proportion, dtype: float64

La expresión **`df_user_reviews["year"].isna().value_counts()`** la utilizamos para contar la cantidad de valores nulos **`(NaN)`** y no nulos en la columna **`year`**, se puede interpretar que  hay **`45,826 `** registros con valores en la columna **`year`** y **`9,487`** registros sin valores **`(valores nulos)`** en la misma columna. 

In [19]:
df_user_reviews["year"].isna().value_counts()

year
False    45826
True      9487
Name: count, dtype: int64

In [20]:
df_user_reviews.isnull().sum()

user_id         0
user_url        0
item_id         0
recommend       0
review          0
year         9487
dtype: int64

llenamos los valores faltantes en la columna **`year`** con la mediana utilizando la función **`impute_year`**, realizando una imputación de datos.

In [29]:
df_user_reviews

,user_id,user_url,item_id,recommend,review,year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,Simple yet with great replayability. In my opi...,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,True,It's unique and worth a playthrough.,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
1,js41637,http://steamcommunity.com/id/js41637,251610,True,I know what you think when you see this title ...,2014
1,js41637,http://steamcommunity.com/id/js41637,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,130,True,if you liked Half life i would really recommen...,NaN
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,70,True,a must have classic from steam definitely wort...,NaN
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,362890,True,this game is a perfect remake of the original ...,NaN
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,273110,True,had so much fun plaing this and collecting res...,NaN


In [30]:
def impute_year (df, column_name):
    # Rellena los valores faltantes con 0
    df[column_name].fillna(0, inplace=True)

    # Convierte la columna a tipo entero
    df[column_name] = df[column_name].astype(int)

# Utiliza la función para rellenar los datos faltantes en la columna 'year'
impute_year(df_user_reviews, 'year')

In [31]:
df_user_reviews.year.isna().sum()

0

In [32]:
# reseter el indice
df_user_reviews.reset_index(drop=True)

,user_id,user_url,item_id,recommend,review,year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,http://steamcommunity.com/id/js41637,251610,True,I know what you think when you see this title ...,2014
4,js41637,http://steamcommunity.com/id/js41637,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...
55308,76561198312638244,http://steamcommunity.com/profiles/76561198312...,130,True,if you liked Half life i would really recommen...,0
55309,76561198312638244,http://steamcommunity.com/profiles/76561198312...,70,True,a must have classic from steam definitely wort...,0
55310,76561198312638244,http://steamcommunity.com/profiles/76561198312...,362890,True,this game is a perfect remake of the original ...,0
55311,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,273110,True,had so much fun plaing this and collecting res...,0


In [33]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55313 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    55313 non-null  object
 1   user_url   55313 non-null  object
 2   item_id    55313 non-null  object
 3   recommend  55313 non-null  object
 4   review     55313 non-null  object
 5   year       55313 non-null  int32 
dtypes: int32(1), object(5)
memory usage: 2.7+ MB


In [34]:
# Filtramos las filas donde 'item_id', 'recommend' y 'review' no son nulos
df_user_reviews = df_user_reviews[df_user_reviews['item_id'].notna() & 
                                  df_user_reviews['recommend'].notna() & 
                                  df_user_reviews['review'].notna()]

In [35]:
df_user_reviews.isnull().sum() 

user_id      0
user_url     0
item_id      0
recommend    0
review       0
year         0
dtype: int64

### Limpieza a la columna **``review``**
Aplicamos análisis de ***sentimiento*** a la columna **`review`** con **NLP** para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [36]:
# Descargar el léxico de VADER
nltk.download('vader_lexicon')

# Crear un analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Función para aplicar análisis de sentimientos a las reseñas
def sentiment(text):
    # Si la reseña está ausente, devolver '1'
    if pd.isnull(text):
        return '1'
    # Obtener las puntuaciones de sentimiento para el texto
    sentiment_scores = sia.polarity_scores(text)
    # Obtener la puntuación compuesta
    compound_score = sentiment_scores['compound']
    # Categorizar el sentimiento basado en la puntuación compuesta
    if compound_score < 0:
        return '0'
    elif compound_score == 0:
        return '1'
    else:
        return '2'

# Aplicar la función a la columna 'review' y reemplazar la columna 'review'
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(sentiment)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\57301\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Debe tomar el valor **`0`** si es ***malo***, **`1`** si es ***neutral*** y **`2`** si es ***positivo***.

In [37]:
# Contar la cantidad de cada valor en la columna 'review'
df_user_reviews['sentiment_analysis'].value_counts()

sentiment_analysis
2    34926
1    10833
0     9554
Name: count, dtype: int64

In [38]:
# Contar la cantidad de cada valor en la columna 'review'
df_user_reviews['sentiment_analysis'].value_counts()

sentiment_analysis
2    34926
1    10833
0     9554
Name: count, dtype: int64

In [39]:
def normalize(df, column_name):
    # La función explode transforma cada elemento de una lista en una fila, replicando los valores del índice.
    exploded_df = df.explode(column_name)
    
    # pd.json_normalize es una función de pandas que normaliza datos semi-estructurados JSON en un dataframe plano.
    normalized_df = pd.json_normalize(exploded_df[column_name])
    
    # reset_index con el parámetro drop=True descarta el índice actual y reemplaza con un índice por defecto (0, 1, 2, ...).
    exploded_df.reset_index(drop=True, inplace=True)
    normalized_df.reset_index(drop=True, inplace=True)
    
    # pd.concat concatena los dataframes a lo largo de un eje. Aquí se está eliminando la columna especificada del dataframe original y luego se concatena con el dataframe normalizado.
    result_df = pd.concat([exploded_df.drop(column_name, axis=1), normalized_df], axis=1)
    
    return result_df

# Uso de la función con el DataFrame df_user_reviews y la columna 'review'
df_user_reviews = normalize(df_user_reviews, 'review')

In [40]:
df_user_reviews.isna().sum()

user_id               0
user_url              0
item_id               0
recommend             0
year                  0
sentiment_analysis    0
dtype: int64

In [41]:
def transform(df):
    # Rellenar los valores nulos con 'NotData'
    df.fillna('NotData', inplace=True)

    # Reemplazar 'NotData' con NaN en la columna 'item_id' y luego rellenar con 0
    df['item_id'] = df['item_id'].replace('NotData', np.nan)
    df['item_id'] = df['item_id'].fillna(0)

    # Convertir la columna sentiment_analysis a int
    df['sentiment_analysis'] = df['sentiment_analysis'].astype(int)

    # Convertir la columna 'item_id' a int
    df['item_id'] = df['item_id'].astype(int)

    # Convertir la columna 'recommend' a bool
    df = df.astype({'recommend': 'bool'})

    # Convertir las columnas 'user_id' y 'user_url' a string
    df = df.astype({'user_id': 'string', 'user_url': 'string'})

    # Ordenar el DataFrame por 'item_id'
    df = df.sort_values('item_id')

    return df

# Aplicar la función transform al DataFrame
df_user_reviews = transform(df_user_reviews.copy())
df_user_reviews.head()

,user_id,user_url,item_id,recommend,year,sentiment_analysis
29853,Hype_R,http://steamcommunity.com/id/Hype_R,10,True,0,2
22237,lanatbeonakeehsasamokoshtan,http://steamcommunity.com/id/lanatbeonakeehsas...,10,True,2014,2
52416,76561198089673598,http://steamcommunity.com/profiles/76561198089...,10,True,2014,1
54946,nottynotty,http://steamcommunity.com/id/nottynotty,10,True,0,1
36294,76561198134580826,http://steamcommunity.com/profiles/76561198134...,10,True,0,2


In [42]:
df_user_reviews.reset_index(drop=True, inplace=True)

In [43]:
df_user_reviews

,user_id,user_url,item_id,recommend,year,sentiment_analysis
0,Hype_R,http://steamcommunity.com/id/Hype_R,10,True,0,2
1,lanatbeonakeehsasamokoshtan,http://steamcommunity.com/id/lanatbeonakeehsas...,10,True,2014,2
2,76561198089673598,http://steamcommunity.com/profiles/76561198089...,10,True,2014,1
3,nottynotty,http://steamcommunity.com/id/nottynotty,10,True,0,1
4,76561198134580826,http://steamcommunity.com/profiles/76561198134...,10,True,0,2
...,...,...,...,...,...,...
55308,76561198075141715,http://steamcommunity.com/profiles/76561198075...,521430,True,0,2
55309,76561198071122396,http://steamcommunity.com/profiles/76561198071...,521570,True,0,0
55310,TfhuAWGscvg,http://steamcommunity.com/id/TfhuAWGscvg,521570,True,0,0
55311,mikeyg74,http://steamcommunity.com/id/mikeyg74,521990,True,0,2


In [44]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55313 entries, 0 to 55312
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             55313 non-null  string
 1   user_url            55313 non-null  string
 2   item_id             55313 non-null  int32 
 3   recommend           55313 non-null  bool  
 4   year                55313 non-null  int32 
 5   sentiment_analysis  55313 non-null  int32 
dtypes: bool(1), int32(3), string(2)
memory usage: 1.5 MB


Convertimos el **`DataFrame`** **`df_users_items`** a un archivo **`Parquet`** utilizando el método **`to_parquet()`**

In [45]:
# eliminar la columna user_url
df_user_reviews.drop(['user_url'], axis=1, inplace=True)

In [46]:
df_user_reviews

,user_id,item_id,recommend,year,sentiment_analysis
0,Hype_R,10,True,0,2
1,lanatbeonakeehsasamokoshtan,10,True,2014,2
2,76561198089673598,10,True,2014,1
3,nottynotty,10,True,0,1
4,76561198134580826,10,True,0,2
...,...,...,...,...,...
55308,76561198075141715,521430,True,0,2
55309,76561198071122396,521570,True,0,0
55310,TfhuAWGscvg,521570,True,0,0
55311,mikeyg74,521990,True,0,2


In [90]:
df_user_reviews.to_parquet('test_reviews.parquet')